In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# ### READ ACCESS ###
user = "read-46560257"
password = "*****"
host = "35.202.133.73"
port = "5432"  # default PostgreSQL port
database = "xano-************"



In [71]:

engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}")

# SQL query
query = "SELECT * FROM public.x2_103_contest_summary_reportingcsv;"



In [72]:
# Read query into DataFrame
df = pd.read_sql(query, con=engine)


In [73]:
df.head(85)

,id,Contest_Name,Format_Contest_Type,Contest_Type,Number_of_Levels,Prize,Total_Views,Total_Joins,Completion_Rate,Number_of_Winners,Reward_Status,Clicks,Average_Time_Spent,Gender,_Location,Client_Name,Age_Breakdown
0,1,Gaming Tournament,non-integrated,private,2,Gaming Champion Title,3200,0,0.0,8,Announced,680,00:00:00,male,Australia,Tayyab,"[""21-25"", ""26-33""]"
1,2,Art Exhibition Contest,integrated,public,1,Artist Recognition Award,1400,0,0.0,5,Announced,280,00:00:00,female,France,Tayyab,"[""21-25"", ""26-34""]"
2,3,Cooking Challenge,non-integrated,private,2,"Master Chef Prize, Culinary Experience Package",750,0,0.0,3,Announced,150,00:00:00,both,Italy,Tayyab,"[""21-25"", ""26-35""]"
3,4,Science Fair Competition,integrated,public,3,"Young Scientist Award, Innovation Recognition",2100,0,0.0,6,Announced,420,00:00:00,male,Germany,Tayyab,"[""21-25"", ""26-36""]"
4,5,Dance Competition,non-integrated,private,1,Dance Excellence Trophy,1650,0,0.0,4,Announced,330,00:00:00,female,Spain,Tayyab,"[""21-25"", ""26-37""]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,81,Scrapbooking Workshop,non-integrated,private,1,"Memory Keeper Award, Creative Documentation Prize",750,0,0.0,3,Pending,150,00:00:00,female,United States,Tayyab,"[""21-25"", ""26-103""]"
81,82,Archery Competition,integrated,public,2,Archery Excellence Award,2150,0,0.0,7,Pending,430,00:00:00,both,South Korea,Tayyab,"[""21-25"", ""26-104""]"
82,83,Calligraphy Brush Painting,non-integrated,private,1,"Brush Painting Master, Calligraphy Art Prize",1050,0,0.0,4,Pending,210,00:00:00,male,China,Tayyab,"[""21-25"", ""26-105""]"
83,84,Frisbee Throwing Contest,integrated,public,2,Disc Sport Champion,1750,0,0.0,5,Pending,350,00:00:00,female,Australia,Tayyab,"[""21-25"", ""26-106""]"


In [74]:
df.shape[0]

107

In [75]:
df.shape[1]

17

In [51]:
df.columns.tolist()

['id',
 'Contest_Name',
 'Format_Contest_Type',
 'Contest_Type',
 'Number_of_Levels',
 'Prize',
 'Total_Views',
 'Total_Joins',
 'Completion_Rate',
 'Number_of_Winners',
 'Reward_Status',
 'Clicks',
 'Average_Time_Spent',
 'Gender',
 'Age_Breakdown',
 '_Location',
 'Client_Name']

## 1. Best-performing contests (based on Total_Views, Total_Joins, Clicks)

In [76]:

# Sort contests by Views, Joins, and Clicks
best_by_views = df.sort_values(by="Total_Views", ascending=False).head(10)
best_by_joins = df.sort_values(by="Total_Joins", ascending=False).head(10)
best_by_clicks = df.sort_values(by="Clicks", ascending=False).head(10)

print("📌 Top 10 Contests by Views")
best_by_views[["id", "Contest_Name", "Total_Views"]]


📌 Top 10 Contests by Views


,id,Contest_Name,Total_Views
17,18,Social Media Influencer Challenge,4200
28,29,Trivia Quiz Championship,4150
40,41,Cybersecurity Challenge,3850
7,8,Fitness Challenge,3500
79,80,Ping Pong Tournament,3250
0,1,Gaming Tournament,3200
26,27,Mobile App Development Challenge,3100
63,64,Rubik's Cube Championship,2850
6,7,Music Festival Contest,2800
34,35,3D Modeling Competition,2750


In [64]:
print("\n📌 Top 10 Contests by Joins")
best_by_joins[["id", "Contest_Name", "Total_Joins"]]


📌 Top 10 Contests by Joins


,id,Contest_Name,Total_Joins
39,40,Carl Simpson,4
57,58,Blair Spencer,4
49,50,Dennis Barnes,4
68,69,free fire,3
2,3,edited contest,2
82,83,lokining,2
73,74,Gula Contest,2
29,30,Tayyab,1
69,70,Play and play,1
61,62,Tayyab,1


In [77]:
print("\n📌 Top 10 Contests by Clicks")
best_by_clicks[["id", "Contest_Name", "Clicks"]]


📌 Top 10 Contests by Clicks


,id,Contest_Name,Clicks
17,18,Social Media Influencer Challenge,840
28,29,Trivia Quiz Championship,830
40,41,Cybersecurity Challenge,770
7,8,Fitness Challenge,700
0,1,Gaming Tournament,680
79,80,Ping Pong Tournament,650
26,27,Mobile App Development Challenge,620
63,64,Rubik's Cube Championship,570
6,7,Music Festival Contest,560
34,35,3D Modeling Competition,550


## 2. Contest types that attract the most users

In [79]:
# Group by Format_Contest_Type and Contest_Type
contest_type_stats = (
    df.groupby(["Format_Contest_Type", "Contest_Type"])
    .agg({
        "Total_Views": "sum",
        "Total_Joins": "sum",
        "Clicks": "sum",
        "Completion_Rate": "mean"
    })
    .reset_index()
    .sort_values(by="Total_Joins", ascending=False)
)

print("📌 Contest Type Performance")
contest_type_stats.head(10)


📌 Contest Type Performance


,Format_Contest_Type,Contest_Type,Total_Views,Total_Joins,Clicks,Completion_Rate
0,integrated,public,86240,0,17138,0.0
1,non-integrated,private,80750,0,16110,0.0


## 3.Relation between Number_of_Levels and Completion_Rate

In [81]:
# Group by Number_of_Levels and compute avg Completion Rate
level_completion = (
    df.groupby("Number_of_Levels")["Completion_Rate"]
    .mean()
    .reset_index()
    .sort_values(by="Number_of_Levels")
)

print("📌 Avg Completion Rate vs Number of Levels")
level_completion


📌 Avg Completion Rate vs Number of Levels


,Number_of_Levels,Completion_Rate
0,0,0.0
1,1,0.0
2,2,0.0
3,3,0.0


## 3. Click-through Rate (CTR: Clicks / Total_Views)

### Business Insight:

- High CTR means users are actively interacting with contest pages, even if they don’t join — this is an early indicator of interest.

- Low CTR indicates weak call-to-action or poor landing page design.

### Actionable Idea:

- Optimize contest pages to improve CTR: better visuals, clearer instructions, or highlighting prizes.

- High CTR but low engagement could signal friction in the registration/joining process.

In [82]:
# Create a new DataFrame for click analysis
click_df = df[['Contest_Name', 'Clicks', 'Total_Views']].copy()

# Ensure numeric columns
click_df['Clicks'] = pd.to_numeric(click_df['Clicks'], errors='coerce').fillna(0)
click_df['Total_Views'] = pd.to_numeric(click_df['Total_Views'], errors='coerce').fillna(0)

# Calculate Click Rate (Click Through Percentage)
click_df['Click_Through_Percentage'] = (click_df['Clicks'] / click_df['Total_Views']) * 100

# Show result
click_df.head()

,Contest_Name,Clicks,Total_Views,Click_Through_Percentage
0,Gaming Tournament,680,3200,21.25
1,Art Exhibition Contest,280,1400,20.00
2,Cooking Challenge,150,750,20.00
3,Science Fair Competition,420,2100,20.00
4,Dance Competition,330,1650,20.00


In [83]:
import pandas as pd

# 1. Create a new DataFrame for time analysis
time_df = df[['Contest_Name', 'Average_Time_Spent', 'Total_Joins']].copy()

# 2. Ensure numeric columns
time_df['Average_Time_Spent'] = pd.to_numeric(time_df['Average_Time_Spent'], errors='coerce').fillna(0)
time_df['Total_Joins'] = pd.to_numeric(time_df['Total_Joins'], errors='coerce').fillna(0)

# 3. Calculate Average Time Spent per Participant
time_df['Avg_Time_Per_Participant_Minutes'] = time_df['Average_Time_Spent'] / time_df['Total_Joins']

# 4. View result
time_df.head(68)

,Contest_Name,Average_Time_Spent,Total_Joins,Avg_Time_Per_Participant_Minutes
0,Gaming Tournament,0.0,0,NaN
1,Art Exhibition Contest,0.0,0,NaN
2,Cooking Challenge,0.0,0,NaN
3,Science Fair Competition,0.0,0,NaN
4,Dance Competition,0.0,0,NaN
...,...,...,...,...
63,Rubik's Cube Championship,0.0,0,NaN
64,Sand Castle Building Contest,0.0,0,NaN
65,Harmonica Playing Contest,0.0,0,NaN
66,Embroidery Art Competition,0.0,0,NaN


In [84]:
import pandas as pd

# 1. Create a new DataFrame for prize analysis
prize_df = df[['Contest_Name', 'Prize', 'Total_Joins']].copy()

# 2. Ensure numeric columns
prize_df['Prize'] = pd.to_numeric(prize_df['Prize'], errors='coerce').fillna(0)
prize_df['Total_Joins'] = pd.to_numeric(prize_df['Total_Joins'], errors='coerce').fillna(0)

# 3. Calculate Prize per Participant
prize_df['Prize_Per_Participant'] = prize_df['Prize'] / prize_df['Total_Joins']

# 4. View result
prize_df.head(58)

,Contest_Name,Prize,Total_Joins,Prize_Per_Participant
0,Gaming Tournament,0.0,0,NaN
1,Art Exhibition Contest,0.0,0,NaN
2,Cooking Challenge,0.0,0,NaN
3,Science Fair Competition,0.0,0,NaN
4,Dance Competition,0.0,0,NaN
5,Debate Championship,0.0,0,NaN
6,Music Festival Contest,0.0,0,NaN
7,Fitness Challenge,0.0,0,NaN
8,Innovation Hackathon,0.0,0,NaN
9,Language Learning Contest,0.0,0,NaN
